# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [25]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path
import json

# Import API key
from api_keys import geoapify_key

In [2]:
print(geoapify_key)

0596d3ca539f4a039ccfee1811226333


In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,pallasovka,50.0500,46.8833,15.19,36,100,5.35,RU,1680430278
1,1,aldan,58.6031,125.3894,-14.34,92,100,0.75,RU,1680430278
2,2,rikitea,-23.1203,-134.9692,26.69,82,69,6.26,PF,1680430279
3,3,albany,42.6001,-73.9662,1.76,72,50,3.13,US,1680430229
4,4,ponta do sol,32.6667,-17.1000,25.95,67,0,2.54,PT,1680430281


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [5]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriImagery",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color = "City"
)
# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [22]:
# Narrow down cities that fit criteria and drop any results with null values
# city_data_df.dtypes

narrow_city_data_df = city_data_df.loc[(city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 21)  & (city_data_df["Cloudiness"] == 0),:]
# narrow_city_data_df.count()

# Drop any rows with null values
narrow_city_data_df = narrow_city_data_df.dropna(how="any")

# Display sample data
narrow_city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
4,4,ponta do sol,32.6667,-17.1000,25.95,67,0,2.54,PT,1680430281
22,22,port alfred,-33.5906,26.8910,23.99,81,0,4.02,ZA,1680430296
34,34,suez,29.9737,32.5263,24.31,22,0,2.60,EG,1680430307
48,48,hermanus,-34.4187,19.2345,24.33,64,0,4.15,ZA,1680430316
73,73,makakilo city,21.3469,-158.0858,23.18,81,0,3.60,US,1680430334
106,106,sao filipe,14.8961,-24.4956,26.84,44,0,3.48,CV,1680430358
118,118,east london,-33.0153,27.9116,26.87,60,0,8.27,ZA,1680430366
200,200,geraldton,-28.7667,114.6000,24.23,69,0,7.72,AU,1680430430
233,233,cape town,-33.9258,18.4232,25.06,67,0,4.12,ZA,1680430220
242,242,bubaque,11.2833,-15.8333,25.29,74,0,4.14,GW,1680430458


### Step 3: Create a new DataFrame called `hotel_df`.

In [23]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = narrow_city_data_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
4,ponta do sol,PT,32.6667,-17.1000,67,
22,port alfred,ZA,-33.5906,26.8910,81,
34,suez,EG,29.9737,32.5263,22,
48,hermanus,ZA,-34.4187,19.2345,64,
73,makakilo city,US,21.3469,-158.0858,81,
106,sao filipe,CV,14.8961,-24.4956,44,
118,east london,ZA,-33.0153,27.9116,60,
200,geraldton,AU,-28.7667,114.6000,69,
233,cape town,ZA,-33.9258,18.4232,67,
242,bubaque,GW,11.2833,-15.8333,74,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [29]:
# Set parameters to search for a hotel
categories = "accommodation.hotel"
radius = 10000
limit = 10

params = {
    "limit":limit,
    "apiKey":geoapify_key,
    "categories":categories    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df.loc[index, "Lat"]
    longitude = hotel_df.loc[index, "Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
ponta do sol - nearest hotel: Estalagem Ponta do Sol
port alfred - nearest hotel: No hotel found
suez - nearest hotel: No hotel found
hermanus - nearest hotel: Aloe guest house
makakilo city - nearest hotel: Embassy Suites by Hilton Oahu Kapolei
sao filipe - nearest hotel: Bela Vista
east london - nearest hotel: No hotel found
geraldton - nearest hotel: Ocean Centre Hotel
cape town - nearest hotel: Townhouse Hotel
bubaque - nearest hotel: Chez Julio
bandar-e lengeh - nearest hotel: Old owner do not like foreigners hotel
vila velha - nearest hotel: Hotel Prainha
tecoanapa - nearest hotel: No hotel found
sainte-rose - nearest hotel: Habitation du Compté
kahului - nearest hotel: Maui Seaside Hotel
pombas - nearest hotel: Aldeia Jerome
safaga - nearest hotel: Dolphin Hotel
nouadhibou - nearest hotel: Hotel valencia
acapulco - nearest hotel: Hotel del Valle
matay - nearest hotel: No hotel found
gat - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
4,ponta do sol,PT,32.6667,-17.1000,67,Estalagem Ponta do Sol
22,port alfred,ZA,-33.5906,26.8910,81,No hotel found
34,suez,EG,29.9737,32.5263,22,No hotel found
48,hermanus,ZA,-34.4187,19.2345,64,Aloe guest house
73,makakilo city,US,21.3469,-158.0858,81,Embassy Suites by Hilton Oahu Kapolei
106,sao filipe,CV,14.8961,-24.4956,44,Bela Vista
118,east london,ZA,-33.0153,27.9116,60,No hotel found
200,geraldton,AU,-28.7667,114.6000,69,Ocean Centre Hotel
233,cape town,ZA,-33.9258,18.4232,67,Townhouse Hotel
242,bubaque,GW,11.2833,-15.8333,74,Chez Julio


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [37]:
%%capture --no-display

# Configure the map plot
hotel_df_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    scale = 4,
    color = "Humidity",
    hover_cols = "all",
    use_index = False
)

# Display the map
hotel_df_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity,City,Country,Hotel Name)